In [22]:
import numpy as np
import pandas as pd
import random
import numpy  as np
import pandas as pd    
from numba import njit                        # For some speed gains                    
from math import radians as DegToRad          # Degrees to radians Conversion
from shapely.geometry import Point            # Used in constraint checking
from shapely.geometry.polygon import Polygon
from tqdm import tqdm                         # For Progressbar
import warnings


In [23]:
from AEP_calculator import fn_fitness_function

# Genetic Algorithm for Turbine Points Layout
1. Gene for this problem would be x,y co-ordinates of one turbine.
2. Chromosome would be 50 x,y co-ordinates for 50 turbines(farmLayout design).
3. There are two constraints in this problem.They are Proximity and perimeter constraints. 
4. Total Aep Value would be calculated based on each chromosome and elite population is found.
5. Crossover Strategy: it takes top 4 layouts provided by sample elite population function as input and randomly picks some 50 points out of them and creates one child layout and adds it to the sample elite population.so that if the child is better than the parents then it will be picked as top layout if not parents will be picked.
6. Mutation Strategy:it takes top 4 layout provided by sample elite population function as input and selects one random layout out of the 4 layouts. from that one layout it retains 30 points and randomly generates 20 new points and adds to it to create one mutated child of 50 points and adds this child to the top 4 layouts.so if the mutated child is better than the parent it will be picked up as top layout if not our sample elite population will be retained in the next iteration.


# Chromosomes

In [24]:
def st_layout():
    
    a=[483,916,1349,1782,2215,2648,3081,3514]
    b=[]
    for i in range(len(a)):
        
        b.append([a[i],50])
        b.append([a[i],3947])
        b.append([50,a[i]])
        b.append([3947,a[i]])
    b.append([50,3947])
    b.append([50,50])
    b.append([3947,50])
    b.append([3947,3947])
    return b



#---------------------------------------------------------------------------------------------------------------------#



def newpoint():
    while(True):
        random = np.random.rand(1,2)*4000
        if (random[0,0]>50 and random[0,0]<3950) and (random[0,1]>50 and random[0,1]<3950):
            return random.tolist()[0]
        
#---------------------------------------------------------------------------------------------------------------------#

def distance(A,B):
    return ((A[0]-B[0])**2+(A[1]-B[1])**2)**0.5

#---------------------------------------------------------------------------------------------------------------------#

def constraint_checker_for_new_point(point,layout: list) -> bool:

    for i in layout:
        if abs(distance(point,i))<=400:
            return False
    return True

#---------------------------------------------------------------------------------------------------------------------#

def fn_chromosome():
    k = st_layout()
    np.random.shuffle(k)
    a=k[:20]
    #a.append(newpoint())
    count=0
    while(len(a)!=50):
        new = newpoint()
        if constraint_checker_for_new_point(new,a):
            a.append(new)
            count+=1
    final = np.array(a).reshape(50,2)
    final = pd.DataFrame(final,columns=["x","y"])
    return final

#---------------------------------------------------------------------------------------------------------------------#

def fn_plot(s):
    plt.scatter(s['x'],s['y'])
    plt.xlabel('x axis')
    plt.ylabel('y axis')
    plt.show()

#---------------------------------------------------------------------------------------------------------------------#

def fn_generate_initial_population(population_size):
    init_population_list = []
    init_population_list = [fn_chromosome() for _ in range(population_size)]
    return init_population_list

# Optimization Function



In [25]:
turb_diam = 100

#Constraints

In [26]:
def checkConstraints(turb_coords, turb_diam):
        """
        -**-THIS FUNCTION SHOULD NOT BE MODIFIED-**-

        Checks if the turbine configuration satisfies the two
        constraints:(i) perimeter constraint,(ii) proximity constraint 
        Prints which constraints are violated if any. Note that this 
        function does not quantifies the amount by which the constraints 
        are violated if any. 

        :called from
            main 

        :param
            turb_coords - 2d np array containing turbine x,y coordinates
            turb_diam   - Diameter of the turbine (m)

        :return
            None. Prints messages.   
        """
        bound_clrnc      = 50
        prox_constr_viol = False
        peri_constr_viol = False

        # create a shapely polygon object of the wind farm
        farm_peri = [(0, 0), (0, 4000), (4000, 4000), (4000, 0)]
        farm_poly = Polygon(farm_peri)

        # checks if for every turbine perimeter constraint is satisfied. 
        # breaks out if False anywhere
        for turb in turb_coords:
            turb = Point(turb)
            inside_farm   = farm_poly.contains(turb)
            correct_clrnc = farm_poly.boundary.distance(turb) >= bound_clrnc
            if (inside_farm == False or correct_clrnc == False):
                peri_constr_viol = True
                break

        # checks if for every turbines proximity constraint is satisfied. 
        # breaks out if False anywhere
        for i,turb1 in enumerate(turb_coords):
            for turb2 in np.delete(turb_coords, i, axis=0):
                if  np.linalg.norm(turb1 - turb2) < 4*turb_diam:
                    prox_constr_viol = True
                    break

        # print messages
        if  peri_constr_viol  == True  and prox_constr_viol == True:
              print('Somewhere both perimeter constraint and proximity constraint are violated\n')
        elif peri_constr_viol == True  and prox_constr_viol == False:
              print('Somewhere perimeter constraint is violated\n')
        elif peri_constr_viol == False and prox_constr_viol == True:
              print('Somewhere proximity constraint is violated\n')
        else: print('Both perimeter and proximity constraints are satisfied !!\n')

        return()


# Evolution

## 1.Crossover

In [27]:
#randomly choose 50 coordinates from top 4 layouts and check if they are satisfying constraints 
def fn_crossover(layout_list):
    layout = layout_list.copy()
    flattened_df = pd.DataFrame(columns = ['x','y'])
    flattened_df =pd.concat(layout_list)
    flattened_df = flattened_df.sample(frac = 1) 
    flattened_df.reset_index(drop = True, inplace =True)
    Xover_coords = flattened_df.to_numpy()
    candidate_index = np.random.randint(low = 0,  high=Xover_coords.shape[0], size=1)
    df = pd.DataFrame(index = [0],columns = ['x','y'])
    df.loc[0] = Xover_coords[candidate_index]
    Xover_coords = np.delete(Xover_coords, candidate_index, axis = 0)
    
                # checks if for every turbine perimeter constraint is satisfied. 
                # breaks out if False anywhere
                # for turb in mutant_coords:
                # turb = Point(turb)
                # inside_farm   = farm_poly.contains(turb)
                # correct_clrnc = farm_poly.boundary.distance(turb) >= bound_clrnc
                # if (inside_farm == False or correct_clrnc == False):
                    #peri_constr_viol = True
                    #break
                # checks if for every turbines proximity constraint is satisfied. 
                # breaks out if False anywhere
                
          
    for i,turb1 in enumerate(Xover_coords):
        
        flag = 0
        #for turb2 in np.delete(df, i, axis=0):
        for j in range(df.shape[0]):
            turb2 = np.array(df.iloc[j])
            #turb2 = np.array(turb2)
            #print(type(turb1))
            #print(turb2)
            if  np.linalg.norm(turb1 - turb2) < 4*turb_diam:
                flag = 1
                break
                
        if flag ==0: 
            
            #df.loc[len(df),'x'] = turb1[0]
            #df.loc[(len(df)-1),'y'] = turb1[1]
            
            df.loc[len(df)] = turb1
            if (len(df)==50):
                break
            
                #crossover_list.append(turb1)
                
                #    np.unique(a, axis=0)


        
        
    #return df
    #layout.append(df)
    return df

## 2.Choose Elite

In [28]:
def fn_sample_elite(new_population,iteration_no):
    AEP_df = pd.DataFrame(columns = ['AEP','iteration_num'])
    
    for i in range (len(new_population)):
        AEP_df.loc[len(AEP_df),'iteration_num'] = iteration_no
        AEP_df.loc[(len(AEP_df)-1),'AEP'] = fn_fitness_function(new_population[i])

     
    elite = AEP_df['AEP'].sort_values(ascending = False).to_frame()[0:4]
    print(elite.index)
    global elite2
    elite2 = AEP_df.sort_values(by = ['AEP'],ascending = False)
    print(elite2)
    
    var = list(elite.index.values)
    
    #print(var)
    #print(elite.index.values)
    elite_layout = [new_population[i] for i in var]
    print('sample elite output type is:',type(elite_layout))
    
    return elite_layout

In [29]:
mutation_probability =0.8
number_of_chromos_in_population =15
number_of_iterations = 25

## 3.Evolute

In [30]:
def fn_evolution(elite_population_layouts,
                  mutation_probability,number_of_chromos_in_population):
    new_pop = pd.DataFrame(index = [0],columns=['x','y'])
    #print(new_pop)
    elite_layout = elite_population_layouts.copy()
    print("elite_layout type is:",type(elite_layout))
    new_pop_iterations = 0
    while (new_pop_iterations < number_of_chromos_in_population):
        #print( iteration)
        #print("fn_evaluation new_pop_iterations", new_pop_iterations)
        #new_pop_iterations += 1 
        if np.random.random() < mutation_probability:
            print("mutation started")
            #print("fn_evaluation if")
            # The candidate (or one of the elite portfolios) is chosen randomly for mutation.
            candidate_id = np.random.randint(low = 0,
                                             high=len(elite_population_layouts))
            
            
            new_pop = fn_chromosome()
            print("lenght of elite layout in mutation before appending is :", len(elite_layout))
            elite_layout.append(new_pop)
            print("lenght of elite layout in mutation after appending is :", len(elite_layout))
            
            
            # We prefer to explore much more in the beginning of the search process to 
            # ensure diversity and avoid local optimum. 
            # As we progress towards the end of search process ( or iterations), 
            # we need to ensure the convergence of the population to a good solution 
            # if not the best. Hence we keep on reducing mutation probability with 
            # each iteraation
            
            mutation_probability = mutation_probability/(new_pop_iterations + 1)
            print("mutation process is taking progress")
            new_pop_iterations = new_pop_iterations+1
        else:
              new_pop = fn_crossover(elite_population_layouts)
              print('type of crossover output is :',type(new_pop) )
        
                
              new_pop_iterations = new_pop_iterations+1
              print("lenght of elite layout in cross over before appending is :", len(elite_layout))
              elite_layout.append(new_pop)
              print("lenght of elite layout in cross over after appending is :", len(elite_layout))
            
    print('type of evaluation output is :',type(elite_layout) )
    return elite_layout

# Initiation of main

In [31]:
def intitiate():

    layout1 = pd.read_csv("layouts/s1.csv")
    layout2 = pd.read_csv("layouts/s2.csv")
    layout3 = pd.read_csv("layouts/s3.csv")
    layout4 = fn_chromosome()

    return [layout1,layout2,layout3,layout4]


In [32]:
def fn_main():
    list_layout = fn_generate_initial_population(4)
    
    new_population = list_layout.copy()
    

    print("type of new population is :",type(new_population))
    
    
    # Run genetic algorithm for number_of_iterations times.
    AEP_df = pd.DataFrame(columns = ['AEP','iteration_num'])
    for i in range(number_of_iterations):
        elite_population = fn_sample_elite(new_population,i)                                               
        
        print("iteration:",i)
        print("length of new population in main function is :",len(elite_population))
        new_population = fn_evolution(elite_population,mutation_probability,number_of_chromos_in_population)

        
        
        

        print_counter = i % 100
        if print_counter == 0:
            print("iteration", i)

    AEP_df = pd.DataFrame(columns = ['AEP'])
    for k in range (len(new_population)):
        AEP_df.loc[len(AEP_df),'AEP'] = fn_fitness_function(new_population[k])
     
    elite = AEP_df['AEP'].sort_values(ascending = False).to_frame()[0:1]
    print(elite)
    print(elite.index.values)
    new_population = new_population[elite.index.values[0]]
    

    return new_population

In [33]:
from AEP_calculator import fn_fitness_function

In [34]:
k = fn_main()

type of new population is : <class 'list'>
Int64Index([3, 2, 1, 0], dtype='int64')
   AEP  iteration_num
0  0.0            0.0
1  0.0            0.0
2  0.0            0.0
3  0.0            0.0
sample elite output type is: <class 'list'>
iteration: 0
length of new population in main function is : 4
elite_layout type is: <class 'list'>
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 4
lenght of elite layout in cross over after appending is : 5
mutation started
lenght of elite layout in mutation before appending is : 5
lenght of elite layout in mutation after appending is : 6
mutation process is taking progress
mutation started
lenght of elite layout in mutation before appending is : 6
lenght of elite layout in mutation after appending is : 7
mutation process is taking progress
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 7
lenght

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 11
lenght of elite layout in cross over after appending is : 12
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 12
lenght of elite layout in cross over after appending is : 13
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 13
lenght of elite layout in cross over after appending is : 14
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 14
lenght of elite layout in cross over after appending is : 15
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 15
lenght of elite layout in cross over after appending is : 16
type of crossover output is : <class 'pandas.

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 18
lenght of elite layout in cross over after appending is : 19
type of evaluation output is : <class 'list'>
Int64Index([18, 8, 1, 2], dtype='int64')
    AEP  iteration_num
0   0.0            5.0
10  0.0            5.0
17  0.0            5.0
16  0.0            5.0
15  0.0            5.0
14  0.0            5.0
13  0.0            5.0
12  0.0            5.0
11  0.0            5.0
9   0.0            5.0
1   0.0            5.0
8   0.0            5.0
7   0.0            5.0
6   0.0            5.0
5   0.0            5.0
4   0.0            5.0
3   0.0            5.0
2   0.0            5.0
18  0.0            5.0
sample elite output type is: <class 'list'>
iteration: 5
length of new population in main function is : 4
elite_layout type is: <class 'list'>
mutation started
lenght of elite layout in mutation before appending is : 4
lenght of elite layout in mutation after ap

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 7
lenght of elite layout in cross over after appending is : 8
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 8
lenght of elite layout in cross over after appending is : 9
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 9
lenght of elite layout in cross over after appending is : 10
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 10
lenght of elite layout in cross over after appending is : 11
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 11
lenght of elite layout in cross over after appending is : 12
type of crossover output is : <class 'pandas.core.

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 14
lenght of elite layout in cross over after appending is : 15
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 15
lenght of elite layout in cross over after appending is : 16
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 16
lenght of elite layout in cross over after appending is : 17
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 17
lenght of elite layout in cross over after appending is : 18
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 18
lenght of elite layout in cross over after appending is : 19
type of evaluation output is : <class 'list'>

...

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 11
lenght of elite layout in cross over after appending is : 12
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 12
lenght of elite layout in cross over after appending is : 13
mutation started
lenght of elite layout in mutation before appending is : 13
lenght of elite layout in mutation after appending is : 14
mutation process is taking progress
mutation started
lenght of elite layout in mutation before appending is : 14
lenght of elite layout in mutation after appending is : 15
mutation process is taking progress
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 15
lenght of elite layout in cross over after appending is : 16
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 18
lenght of elite layout in cross over after appending is : 19
type of evaluation output is : <class 'list'>
Int64Index([18, 8, 1, 2], dtype='int64')
    AEP  iteration_num
0   0.0           17.0
10  0.0           17.0
17  0.0           17.0
16  0.0           17.0
15  0.0           17.0
14  0.0           17.0
13  0.0           17.0
12  0.0           17.0
11  0.0           17.0
9   0.0           17.0
1   0.0           17.0
8   0.0           17.0
7   0.0           17.0
6   0.0           17.0
5   0.0           17.0
4   0.0           17.0
3   0.0           17.0
2   0.0           17.0
18  0.0           17.0
sample elite output type is: <class 'list'>
iteration: 17
length of new population in main function is : 4
elite_layout type is: <class 'list'>
mutation started
lenght of elite layout in mutation before appending is : 4
lenght of elite layout in mutation after a

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 7
lenght of elite layout in cross over after appending is : 8
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 8
lenght of elite layout in cross over after appending is : 9
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 9
lenght of elite layout in cross over after appending is : 10
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 10
lenght of elite layout in cross over after appending is : 11
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 11
lenght of elite layout in cross over after appending is : 12
mutation started
lenght of elite layout in mutatio

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 14
lenght of elite layout in cross over after appending is : 15
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 15
lenght of elite layout in cross over after appending is : 16
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 16
lenght of elite layout in cross over after appending is : 17
mutation started
lenght of elite layout in mutation before appending is : 17
lenght of elite layout in mutation after appending is : 18
mutation process is taking progress
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 18
lenght of elite layout in cross over after appending is : 19
type of evaluation output is : <class 'list'>
Int64Index([18, 8,

type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 4
lenght of elite layout in cross over after appending is : 5
mutation started
lenght of elite layout in mutation before appending is : 5
lenght of elite layout in mutation after appending is : 6
mutation process is taking progress
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 6
lenght of elite layout in cross over after appending is : 7
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 7
lenght of elite layout in cross over after appending is : 8
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght of elite layout in cross over before appending is : 8
lenght of elite layout in cross over after appending is : 9
type of crossover output is : <class 'pandas.core.frame.DataFrame'>
lenght

In [ ]:
fn_plot(k)

In [ ]:
fn_fitness_function(k)

# Submission

In [ ]:
k.to_csv("submission.csv",index=False)